In [ ]:
import pandas as pd
import numpy as np
from time import time
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import randint, uniform

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, auc, average_precision_score

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import time

In [ ]:
import keras
import bctools as bc
from bayes_opt import BayesianOptimization, UtilityFunction
from hyperopt import hp, fmin, tpe

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
precision_recall_curve()

### Import and explore data

In [ ]:
df = pd.read_csv("./data/train.csv")

In [ ]:
df.head()

In [ ]:
df.describe(include="all")

In [ ]:
df.Surname.value_counts()

In [ ]:
df.CustomerId.value_counts()

In [ ]:
# Remove unique IDs from the table as they serve no purpose
df.drop(columns=["id", "CustomerId", "Surname"], inplace=True)
df.head()

In [ ]:
# Check size
df.shape

In [ ]:
# Check label distribution
df.Exited.value_counts()

In [ ]:
print( "Number of positive samples in training data: {} ({:.2f}% of total)".format(df.Exited.value_counts()[1], 100 * (df.Exited.value_counts()[1]/ df.shape[0])))

In [ ]:
df.info()

### Visualisation

In [ ]:
fig = px.histogram(df, x="CreditScore", color="Exited", marginal="box", hover_data=df.columns, height=500, width=900)
fig.show()

In [ ]:
fig = px.histogram(df, x="Balance", color="Exited", marginal="box", hover_data=df.columns, height=500, width=900)
fig.show()

In [ ]:
fig = px.histogram(df, x="EstimatedSalary", color="Exited", marginal="box", hover_data=df.columns, height=500, width=900)
fig.show()

## Preprocessing and data engineering

#### Transform data

In [ ]:
df["BalanceLog"] = np.log(df.Balance)

In [ ]:
df_ = df.copy()
df_["Balance"] = df.BalanceLog

In [ ]:
fig = px.histogram(df, x="BalanceLog", color="Exited", marginal="box", hover_data=df.columns, height=500, width=900)
fig.show()

#### Change data type

In [ ]:
df.head()

In [ ]:
convert_dict = {"Geography":"category",
               "Gender":"category",
               "HasCrCard":"category",
               "IsActiveMember":"category",}

In [ ]:
df = df.astype(convert_dict)
df.info()

#### Train test split

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
df.iloc[:,:10].head()

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:10], df.iloc[:,10].astype("float32"), test_size=0.2, random_state=42, stratify=df.iloc[:,10].astype("float32"))

In [ ]:
bins = np.bincount(y_train)

In [ ]:
bins_test = np.bincount(y_test)

In [ ]:
X_train.head()

In [ ]:
X_train.describe(include="all")

In [ ]:
X_test.describe(include="all")

#### Preprocessing pipeline


In [ ]:
categorical_features = X_train.select_dtypes(include=["category"]).columns

In [ ]:
numerical_features = X_test.select_dtypes(exclude=["category"]).columns

In [ ]:
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('ordinal', OneHotEncoder())])

preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_features),
                                              ('cat', categorical_transformer, categorical_features)])

In [ ]:
preprocessor.fit(X_train)

In [ ]:
X_train_ = preprocessor.transform(X_train)
X_test_ = preprocessor.transform(X_test)

## Modelling
This section models and perform hyperparameter tuning for 4 tree based models. <br>
The hyperparamter tuning optimizer is Bayesian Optimization

In [ ]:
accuracy = {}
speed = {}

In [ ]:
# baysian optimisation function


#### Gradient boosted classifer

In [ ]:
def gbm_cl_bo(max_depth, max_features, learning_rate, n_estimators, subsample):
    params_gbm = {}
    params_gbm['max_depth'] = round(max_depth)
    params_gbm['max_features'] = max_features
    params_gbm['learning_rate'] = learning_rate
    params_gbm['n_estimators'] = round(n_estimators)
    params_gbm['subsample'] = subsample
    scores = cross_val_score(GradientBoostingClassifier(random_state=123, **params_gbm),
                             X_train_, y_train, scoring='average_precision', cv=5).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_gbm ={
    'max_depth':(3, 10),
    'max_features':(0.8, 1),
    'learning_rate':(0.01, 1),
    'n_estimators':(80, 150),
    'subsample': (0.8, 1)
}
gbm_bo = BayesianOptimization(gbm_cl_bo, params_gbm, random_state=111)
gbm_bo.maximize(init_points=10, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))


In [ ]:
params_gbm = gbm_bo.max['params']
params_gbm['max_depth'] = round(params_gbm['max_depth'])
params_gbm['n_estimators'] = round(params_gbm['n_estimators'])
params_gbm

In [ ]:
model_gbm = GradientBoostingClassifier(random_state=123, **params_gbm)
model_gbm.fit(X_train_, y_train)

#### XGBoost (eXtreme Gradient Boosting)

In [ ]:
def xgb_cl_bo(max_depth,learning_rate, n_estimators):
    params_xgb = {}
    params_xgb['max_depth'] = round(max_depth)
    params_xgb['learning_rate'] = learning_rate
    params_xgb['n_estimators'] = round(n_estimators)
    scores = cross_val_score(XGBClassifier(random_state=123, **params_xgb),
                             X_train_, y_train, scoring='average_precision', cv=5).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
param_dist_xgb = {
    'n_estimators': (50, 200),
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.2),
}
xgb_bo = BayesianOptimization(xgb_cl_bo, param_dist_xgb, random_state=111)
xgb_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

In [ ]:
params_xgb = xgb_bo.max['params']
params_xgb['max_depth'] = round(params_xgb['max_depth'])
params_xgb['n_estimators'] = round(params_xgb['n_estimators'])
params_xgb

In [ ]:
model_xgb = XGBClassifier(random_state=123, **params_xgb)
model_xgb.fit(X_train_, y_train)

#### LightGBM

In [ ]:
def lgbm_cl_bo(num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree, reg_alpha, reg_lambda, max_depth,learning_rate, n_estimators):
    params_lgbm = {}
    params_lgbm['num_leaves'] = round(num_leaves)
    params_lgbm['min_child_samples'] = round(min_child_samples)
    params_lgbm['min_child_weight'] = round(min_child_weight)
    params_lgbm['subsample'] = subsample
    params_lgbm['colsample_bytree'] = colsample_bytree
    params_lgbm['reg_alpha'] = round(reg_alpha)
    params_lgbm['reg_lambda'] = round(reg_lambda)
    params_lgbm['max_depth'] = round(max_depth)
    params_lgbm['learning_rate'] = learning_rate
    params_lgbm['n_estimators'] = round(n_estimators)

    
    scores = cross_val_score(LGBMClassifier(random_state=42, **params_lgbm),
                             X_train_, y_train, scoring='average_precision', cv=5, verbose=0).mean()
    score = scores.mean()
    return score

param_dist_lgbm ={'num_leaves': (6, 50), 
             'min_child_samples': (100, 500), 
             'min_child_weight': (1, 1e3),
             'subsample': (0.2, 0.8), 
             'colsample_bytree': (0.4, 0.6),
             'reg_alpha': (0, 100),
             'reg_lambda': (0, 100),
            'n_estimators': (50, 200),
            'max_depth': (3, 10),
            'learning_rate': (0.01, 0.2),}

start = time.time()

lgbm_bo = BayesianOptimization(lgbm_cl_bo, param_dist_lgbm, random_state=111)
lgbm_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

In [ ]:
params_lgbm = lgbm_bo.max['params']
params_lgbm['max_depth'] = round(params_lgbm['max_depth'])
params_lgbm['n_estimators'] = round(params_lgbm['n_estimators'])
params_lgbm['num_leaves'] = round(params_lgbm['num_leaves'])
params_lgbm['min_child_samples'] = round(params_lgbm['min_child_samples'])
params_lgbm['min_child_weight'] = round(params_lgbm['min_child_weight'])
params_lgbm['reg_alpha'] = round(params_lgbm['reg_alpha'])
params_lgbm['reg_lambda'] = round(params_lgbm['reg_lambda'])
params_lgbm

In [ ]:
model_lgbm = LGBMClassifier(random_state=42, **params_lgbm)
model_lgbm.fit(X_train_, y_train)

#### CatBoost

In [ ]:
def catb_cl_bo(depth,learning_rate, iterations, min_child_samples, l2_leaf_reg):
    params_catb = {}
    params_catb['depth'] = round(depth)
    params_catb['learning_rate'] = learning_rate
    params_catb['iterations'] = round(iterations)
    params_catb['l2_leaf_reg'] = l2_leaf_reg
    params_catb['min_child_samples'] = round(min_child_samples)
    
    scores = cross_val_score(CatBoostClassifier(silent=True,random_state=123, **params_catb),
                             X_train_, y_train, scoring='average_precision', cv=5).mean()
    score = scores.mean()
    return score
    
# Run Bayesian Optimization
start = time.time()
param_dist_catb = {
    'iterations': (50, 1000),
    'depth': (3, 10),
    'learning_rate': (0.01, 0.2),
    'min_child_samples':(1, 32),
    'l2_leaf_reg':(0.5, 5.0),
}
catb_bo = BayesianOptimization(catb_cl_bo, param_dist_catb, random_state=111)
catb_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))


In [ ]:
params_catb = catb_bo.max['params']
params_catb['depth'] = round(params_catb['depth'])
params_catb['iterations'] = round(params_catb['iterations'])
params_catb['min_child_samples'] = round(params_catb['min_child_samples'])
params_catb

In [ ]:
model_catb = CatBoostClassifier(silent=True,random_state=123, **params_catb)
model_catb.fit(X_train_, y_train)

#### Neural Network

In [ ]:
# neural Network class weights
weight_for_0 = 1.0 / bins[0]
weight_for_1 = 1.0 / bins[1]

print(weight_for_0, weight_for_1)

In [ ]:
del(model)

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=X_train_.shape[1:]),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

In [ ]:
metrics = [
    keras.metrics.Accuracy(name="accuracy"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics, 
)

callbacks = [keras.callbacks.ModelCheckpoint("./keras_model/churn_model_at_epoch.keras"), keras.callbacks.EarlyStopping(monitor='loss', patience=3)]
class_weight = {0: weight_for_0, 1: weight_for_1}

In [ ]:
model.fit(
    X_train_,
    y_train,
    batch_size=2048,
    epochs=30,
    verbose=1,
    callbacks=callbacks,
    validation_data=(X_test_, y_test),
    #class_weight=class_weight,
)

#### Model predictions

In [ ]:
preds_nn = model.predict(X_test_)
#preds_gbm = model_gbm.predict_proba(X_test_)
preds_xgb = model_xgb.predict_proba(X_test_)
preds_lgbm = model_lgbm.predict_proba(X_test_)
preds_catb = model_catb.predict_proba(X_test_)

In [ ]:
def getPlotingData(y_true, y_score):
    precision, recall, thresholds = precision_recall_curve(y_true, y_score)
    f1_scores = 2*recall*precision/(recall+precision)
    ls = []
    for i in range(len(thresholds)):
        ls.append(f"Threshold: {thresholds[i]:.5f}\n f1 score: {f1_scores[i]:.4f}")

    return precision, recall, thresholds, ls
    

In [ ]:
precision_nn, recall_nn, thresholds_nn, hovertxt_nn = getPlotingData(y_test, preds_nn)
#precision_gbm, recall_gbm, thresholds_gbm, hovertxt_gbm = getPlotingData(y_test, preds_gbm[:, 1])
precision_xgb, recall_xgb, thresholds_xgb, hovertxt_xgb = getPlotingData(y_test, preds_xgb[:, 1])
precision_lgbm, recall_lgbm, thresholds_lgbm, hovertxt_lgbm = getPlotingData(y_test, preds_lgbm[:, 1])
precision_catb, recall_catb, thresholds_catb, hovertxt_catb = getPlotingData(y_test, preds_catb[:, 1])

In [ ]:
fig = go.Figure()
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
auc_score = auc(recall_nn,precision_nn)
name = f"NN (AURCP={auc_score:.4f})"
fig.add_trace(go.Scatter(x=recall_nn, y=precision_nn, name=name, mode='lines', hovertext=hovertxt_nn))
#auc_score = auc(recall_gbm,precision_gbm)
#name = f"GBM (AURCP={auc_score:.4f})"
#fig.add_trace(go.Scatter(x=recall_gbm, y=precision_gbm, name=name, mode='lines', hovertext= hovertxt_gbm))
auc_score = auc(recall_xgb,precision_xgb)
name = f"XGB (AURCP={auc_score:.4f})"
fig.add_trace(go.Scatter(x=recall_xgb, y=precision_xgb, name=name, mode='lines', hovertext=hovertxt_xgb))
auc_score = auc(recall_lgbm,precision_lgbm)
name = f"LGBM (AURCP={auc_score:.4f})"
fig.add_trace(go.Scatter(x=recall_lgbm, y=precision_lgbm, name=name, mode='lines', hovertext=hovertxt_lgbm))
auc_score = auc(recall_catb,precision_catb)
name = f"CATB (AURCP={auc_score:.4f})"
fig.add_trace(go.Scatter(x=recall_catb, y=precision_catb, name=name, mode='lines', hovertext=hovertxt_catb))

fig.update_layout(
    title="Area Under Precision Recall Curve for Multiple Models",
    xaxis_title='Recall',
    yaxis_title='Precision',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=800, height=600
)

fig.show()

Catboost is the best model